# [NTDS'18] Final project
[ntds'18]: https://github.com/mdeff/ntds_2018

* Team: `4` ([github repo](https://github.com/swouf/ntds_IMDb_team4))
* Dataset: `IMDb Films and Crew`

### Authors :
+ [Julien Berger](mailto:juliengaetan.berger@epfl.ch)
+ [Jérémy Jayet](mailto:jeremy.jayet@epfl.ch)
+ [Hana Samet](mailto:hana.samet@epfl.ch)
+ [Mathieu Shiva](mailto:mathieu.shiva@epfl.ch)

## 0 - Import packages and data

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from scipy import sparse
import scipy.sparse.linalg
from matplotlib import pyplot as plt
from pyunlocbox import functions, solvers
import pygsp as pg
import networkx as nx
import logging
import json
import sys
import seaborn as sns
import collections

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

import src.load_data as ldata
import src.genre_graph as genre_graph
import src.test_success as tsuccess

## 1 - Clean the initial dataset and explore it

The initial dataset is composed of two csv files. One contains informations about the movies, as seen in the cell below. It initialy contains 4803 movies, and 20 columns

In [ ]:
initial_movies = pd.read_csv('./data/tmdb_5000_movies.csv')
print(initial_movies.shape)
initial_movies.head(3)

The second csv file contains informations about the cast and the crew of all 4803 movies. The information is stored in json columns.

In [ ]:
initial_credits = pd.read_csv('./data/tmdb_5000_credits.csv')
print(initial_credits.shape)
initial_credits.head(3)

As both csv files contain columns that have a json format, we first have to expend them. This is achieved by our function load_dataframe, which returns a cleaner version of the movies dataframe and a "people" dataframe that contains an extended list of the people in the dataset.

In [ ]:
(movies,people) = ldata.load_dataframes();

For the movies dataframe, we removed movies that were missing information about their revenue or their budget, thus we only have 3229 movies left. We expended the genre column which was previously a json column, and assigned and id to each genre (present in the column "genre"). The name of the genre is kept in the column "genres_names". We added the return on investment under the "ROI" column ($ROI=\frac{revenue-budget}{budget}$), and a success column which has a value of 1 if the ROI of the movie is superior to 0.5 and a value of 0 otherwise. We also changed the ID of the movies, by changing to values between 0 and 4802.

In [ ]:
print(movies.shape)
movies.head(3)

For the people dataframe, we expended the json format and made a list that contains 1 row for each role or job. Their are 193655 rows (combination of actor or crew and their job in the movie), but only 87106 unique people (for example if Bruce Willis played 10 different roles, their will be 10 different rows for Bruce Willis, but each one will have a unique id.).

In [ ]:
print(people.shape)
print(people['name'].unique().shape)
people.head(3)

We then realized that only a few of the ROI had high values (above 10), while the rest of the movies had values very close to 1, which created issues when we wanted to plot them. We thus filtered them.

In [ ]:
print('The 10 largest ROI are \n {}'.format(movies['ROI'].nlargest(10)))
print('The number of ROI values which are larger than 10 is {}'.format(movies['ROI'][movies['ROI']>10].count()))
print('The average ROI is {}'.format(movies['ROI'].mean()))
print('The median ROI is {}'.format(movies['ROI'].median()))

In [ ]:
filter_ROI=movies['ROI'].copy()
filter_ROI[filter_ROI>10]=10
movies['ROI']=filter_ROI

We created a list that allowed to make the conversion between the movies genres and the genre ID that we assigned them

In [ ]:
list_genres=movies[['genres_names','genres']].copy()
list_genres=list_genres.drop_duplicates()
list_genres=list_genres.set_index('genres') 
list_genres

We plotted at the histograms for the budget, the revenue and the ROI, and observed that they all had tails-like distributions. Additionaly, for the ROI, it is important to note that a fair amount of movies havenegative values, meaning that had revenues that were smaller than their budget.

In [ ]:
fig=plt.figure(figsize = (5,5))
movies.budget.plot(kind='hist', bins = 20)
plt.title('Budget of all movies')
plt.show()
fig.savefig('./pictures/budgets.png')

fig2=plt.figure(figsize = (5,5))
movies.revenue.plot(kind='hist', bins = 80)
plt.title('Revenue of all movies')
plt.axis([-0.05e9,1e9,0,1400])
plt.show()
fig2.savefig('./pictures/revenue.png')

fig3=plt.figure(figsize = (5,5))
movies.ROI.plot(kind='hist', bins = 40)
plt.title('ROI of all movies')
plt.axis([-1.5,10,0,250])
plt.show()
fig3.savefig('./pictures/roi.png')

We then looked at the different genres that we had, and looked at their repartition. We see that although their are 20 genres present in the dataset, only 7 genres have more than 100 movies. Drama, Comedy and Action represent 61% of the movies.

In [ ]:
genres_count=movies['genres_names'].value_counts()
print(genres_count)

In [ ]:
fig=plt.figure(figsize = (10,10))
plt.pie(genres_count, autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',labeldistance=1.1,startangle=0,textprops={'fontsize': 15})
plt.title('Percentage of the genres in the whole dataset',fontsize=20)
plt.legend(genres_count,labels=movies['genres_names'].value_counts().index, bbox_to_anchor=(0.9,0.80), bbox_transform=plt.gcf().transFigure,prop={'size': 14})
plt.show()
fig.savefig('./pictures/pie.png')

We looked in details at the different production companies, the number of movies that they were responsible and their ROI.

In [ ]:
production_companies = []
movies.apply(lambda row: [x.update({'id': row['id']}) for x in row['production_companies']], axis=1);

movies.production_companies.apply(lambda x: production_companies.extend(x))
production_companies = pd.DataFrame(production_companies)

production_companies['type'] = 'production_companies'
production_companies=production_companies.drop_duplicates('id')
production_companies['index']=production_companies['id']
production_companies=production_companies.set_index('id') 

list_of_companies=production_companies['name'].unique()
nb_companies=len(list_of_companies) # there are 753 production companies
companies_names=production_companies['name'].copy()

movies['companies_names']=companies_names
factorized_names_companies = pd.factorize(movies['companies_names'])[0]

movies['production_companies']=factorized_names_companies

#the list that makes the conversion between companies names and their id
list_companies=movies[['companies_names','production_companies']].copy()
list_companies=list_companies.drop_duplicates()
list_companies['id']=list_companies['production_companies']
list_companies=list_companies.set_index('production_companies') 

data_companies=movies['companies_names'].value_counts().tolist()

data_companies=pd.DataFrame(data_companies)
list_companies['number_film']=data_companies
list_companies
data_companies_relevent=movies['companies_names'].value_counts()
data_companies_relevent=data_companies_relevent[(data_companies_relevent> 30)]
data_companies_relevent

list_companies=list_companies.assign(ROI_TOT=0)
   
for i in range(0,753):
    list_companies.at[i, 'ROI_TOT'] =sum(movies[movies['production_companies'] == i]['ROI'])


print('Their are {} different production companies'.format(list_companies.shape[0]))
list_companies.head(3)


Although their are 754 production companies in the dataset, we see that only four of them are responsible for 52% of the movies

In [ ]:
plt.figure(figsize = (10,10))
plt.pie(data_companies_relevent, autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',labeldistance=1.1,startangle=0,textprops={'fontsize': 15})
plt.title('Percentage of the companies in the whole dataset',fontsize=20)
plt.legend(data_companies_relevent,labels=movies['companies_names'].value_counts().index, bbox_to_anchor=(0.9,0.80), bbox_transform=plt.gcf().transFigure,prop={'size': 14})
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(list_companies['id'],list_companies['ROI_TOT'])
plt.scatter(list_companies['id'],list_companies['number_film'])
plt.title('The total ROI for each production company and the respective number of movies ',fontsize=10);
plt.xlabel('Production companies')
plt.ylabel('Number of movies ')
plt.show()

In [ ]:
list_companies_succesful=list_companies[list_companies['ROI_TOT'] >80]

plt.figure(figsize=(15, 10))

plt.scatter(list_companies_succesful['id'],list_companies_succesful['ROI_TOT'])
plt.scatter(list_companies_succesful['id'],list_companies_succesful['number_film'])
plt.title('The total ROI for the most successful  production company and the respective number of movies ',fontsize=10);
plt.xlabel('Production companies')
plt.ylabel('Number of movies ')
plt.show()

## 2 - Create the feature matrix and a graph using the actors

We wanted to create a graph of movies by connecting them if they had similar actors. For that we created a function that uses the movies and the people dataframes, and return a dataframe with one row for each person. The first three columns contain the ID of the person, their name and the average rating of all their movies. Then their are 3229 columns, one for each movie, and the value of a cell is equal to 1 if the person worked on this movie (either as a cast or a crew), and 0 if the person did not. For this, we only kept the people that had worked on at least 5 movies, leaving us with 7940 people only. This was done to encure that our  graph would not be fully connected.

In [ ]:
#Used to create a new feature matrix
#As the function takes some time to run, we saved the feature matrix and are simply loading it.
#features_actors=ldata.create_features(movies,people);
features=pd.read_csv('./data/test_actors_crew.csv')
features = features.drop(features.columns[0],axis=1)
print(features.shape)
features.head(3)

Then we created an adjacency matrix that links movies with actors and crew in common, however they need to have more than 1 person in common, to avoid a fully conected graph. The weight of the edges is equal to the number of people in common. We also did the same thing but using only the actors and not the crew. Both adjacency matrices are shown below.

In [ ]:
#As both function used to create the adjacency take some time to run, we saved the adjacency and simply load it.

#Used to load the feature dataframe from a file, remove the useless columns and transpose it.
#features_movies=ldata.load_features();

#Used to create the adjacency matrix from the features
#As the function takes some time to run, we saved the adjacency matrix and are simply loading it.
#adjacency_test=ldata.make_adjacency_from_feature_matrix(features_movies)


plt.figure(figsize=(20, 10))

plt.subplot(121)
adjacency_actors_crew=np.load('./data/adjacency_actors_crew.npy')
plt.spy(adjacency_actors_crew, markersize=0.1)
plt.title('adjacency matrix using actors and crew');

plt.subplot(122)
adjacency_actors_crew=np.load('./data/adjacency_actors_only.npy')
plt.spy(adjacency_actors_crew, markersize=0.1)
plt.title('adjacency matrix using only the actors');
plt.show()

For the rest of the project, we decided to use the adjacency matrix with actors and the crew, however as both adjacency matrices were saved they can be easily swapped if needed. We then created a graph using this adjacency matrix. As shown below, there are many unconnected movies (338 connected componenents in total), so we decided to use the giant component for the next steps.

In [ ]:
#We decided to use the adjacency with actors and the crew, but the one with actors only can be used if needed.
#adjacency_actors=np.load('./data/adjacency_actors_only.npy')
adjacency_actors=np.load('./data/adjacency_actors_crew.npy')

G = nx.from_numpy_matrix(adjacency_actors)
print('The number of connected components is {}'.format(nx.number_connected_components(G)))
coords = nx.spring_layout(G,k=0.03)  # Force-directed layout.
labels = movies['genres'].iloc[np.sort(nx.nodes(G))]

fig=plt.figure(figsize=(15, 10))
im=nx.draw_networkx_nodes(G, coords, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
nx.draw_networkx_edges(G, coords, alpha=0.1, width=0.7)
plt.title('Movies connected by at least 2 common actors, the colors represent the different movie genres')
plt.colorbar(im);
fig.savefig('./pictures/graph1.png')

We plot the giant component and see that it contains 2883 movies. We also plot it with only the three most represented movie genres (Action, Drama and Comedy)

In [ ]:
Gc = max(nx.connected_component_subgraphs(G), key=len)
coords_Gc = nx.spring_layout(Gc,k=0.03)  # Force-directed layout.

print('The number of nodes is is {}'.format(Gc.number_of_nodes()))
labels = movies['genres'].iloc[np.sort(nx.nodes(Gc))]
fig=plt.figure(figsize=(15, 10))
im=nx.draw_networkx_nodes(Gc, coords_Gc, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
nx.draw_networkx_edges(Gc, coords_Gc, alpha=0.1, width=0.7)
plt.title('Giant component of the movies connected by at least 2 common actors')
plt.colorbar(im);
fig.savefig('./pictures/graph1.png')

#We mask the movies which do no belong to the 3 most represented genres
masked_labels=labels.copy()
masked_labels[masked_labels<30]=30
masked_labels[labels==0]=0
masked_labels[labels==5]=5
masked_labels[labels==8]=8
masked_labels[masked_labels>10]=np.nan

#action = 0 = blue
#drama = 5 = red
#comedy = 8= green

fig2=plt.figure(figsize=(15, 10))
im=nx.draw_networkx_nodes(Gc, coords_Gc, node_size=10,node_color=masked_labels, cmap='brg',vmin=0, vmax=8)
nx.draw_networkx_edges(Gc, coords_Gc, alpha=0.1, width=0.7)
plt.title('Movies connected by at least 2 common actors with only action (blue), drama(red) and comedy (green)')
plt.colorbar(im);
fig2.savefig('./pictures/graph2.png')


We can then use this graph to represen different signal, for example the average rating of each movie, or the success rate.
As seen on both graphs below, there are no distinct communities of movies that are grouped with the same rating or success rate.

In [ ]:
labels = movies['vote_average'].iloc[np.sort(nx.nodes(Gc))]
plt.figure(figsize=(15, 10))
im=nx.draw_networkx_nodes(Gc, coords_Gc, node_size=10,node_color=labels, cmap='PiYG')
nx.draw_networkx_edges(Gc, coords_Gc, alpha=0.1, width=0.7)
plt.title('Giant component graph of the vote average')
plt.colorbar(im);

In [ ]:
labels = movies['success'].iloc[np.sort(nx.nodes(Gc))]
plt.figure(figsize=(15, 10))
im=nx.draw_networkx_nodes(Gc, coords_Gc, node_size=10,node_color=labels, cmap='PiYG')
nx.draw_networkx_edges(Gc, coords_Gc, alpha=0.1, width=0.7)
plt.title('Giant component graph of the vote average')
plt.colorbar(im);

## 3 - Separation into decades

Our initial idea was to look at the evolution of the movie industry throughout the years. For that, we split the dataset into 6 different subset, one from each decade, from 1960 to 2020. As there was only a few movies from before 1960 in the dataset, we decided not to use them.

In [ ]:
numberOfDecades = 6

moviesFilteredByYears = []

for i in range(numberOfDecades):
    decade = 1960 + i*10
    decadeEnd = decade + 10
    moviesFilteredByYears.append(ldata.filter_movies_by_years(movies, pd.to_datetime(f'{decade}-01-01').date(), pd.to_datetime(f'{decadeEnd}-01-01').date()))

We created the same graphs, connecting movies that have actors or crew in common, but for each decade. This was achieved by keeping only the desired rows and columns of the complete adjacency matrix. Below are shown the 6 different adjacency matrices, the corresponding complete graphs and giant component graphs. We see that the number of movies increases with each deacde (with the exception of 2010-2020, since it is not over yet)

In [ ]:
adjacency=np.load('./data/adjacency_actors_crew.npy')
moviesFilteredByYears
for i in range(numberOfDecades):
    
    decade = 1960 + i*10
    decadeEnd = decade + 10
    
    #We filter the entire adjacency matrix, by keeping only the rows and columns corresponding to the movies in the decade.
    id_to_keep=moviesFilteredByYears[i].index
    adjacency_id= movies.loc[:, ['id']]
    adjacency_id['id_order']=np.arange(movies.shape[0])
    adjacency_id=adjacency_id.loc[adjacency_id['id'].isin(id_to_keep)]
    
    lines_to_keep=adjacency_id['id_order']
    adjacency_tot=adjacency.copy()
    adjacency_decade_filtered=adjacency_tot[lines_to_keep,:]
    adjacency_decade_filtered=adjacency_decade_filtered[:,lines_to_keep]
    
    plt.figure(figsize=(28, 7))
    plt.subplot(131)
    plt.spy(adjacency_decade_filtered, markersize=1)
    plt.title('Adjacency for the years {} to {}'.format(decade, decadeEnd))
    
    G_decade = nx.from_numpy_matrix(adjacency_decade_filtered)
    coords_decade = nx.spring_layout(G_decade,k=0.03)  
    labels = moviesFilteredByYears[i]['genres'].iloc[np.sort(nx.nodes(G_decade))]
    
    plt.subplot(132)
    im=nx.draw_networkx_nodes(G_decade, coords_decade, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
    nx.draw_networkx_edges(G_decade, coords_decade, alpha=0.1, width=0.7)
    plt.title('Complete graph of common actors, with {} connected components'.format(nx.number_connected_components(G_decade)))
    plt.colorbar(im);
    
    Gc_decade = max(nx.connected_component_subgraphs(G_decade), key=len)
    nb_edges_gc=Gc_decade.number_of_edges()
    nb_nodes_gc=Gc_decade.number_of_nodes()
    coords_decade_Gc = nx.spring_layout(Gc_decade,k=0.03)  
    labels = moviesFilteredByYears[i]['genres'].iloc[np.sort(nx.nodes(Gc_decade))]
    
    plt.subplot(133)
    im=nx.draw_networkx_nodes(Gc_decade, coords_decade_Gc, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
    nx.draw_networkx_edges(Gc_decade, coords_decade_Gc, alpha=0.1, width=0.7)
    plt.title('Giant component graph, with {} edges and {} nodes'.format(nb_edges_gc,nb_nodes_gc))
    plt.colorbar(im);

Then, we used the budget and the revenue of the movies, to make additional adjacency matrices for each decade. We built them in our function "make_budget_based_adjacency", using similarities between the budget and genres. Then all the graphs and adjacency matrices are stocked for later use. We also show the weights distribution that are used to build the 6 different adjacency matrices.

In [ ]:
#We append the information for each decade
adjacenciesFilteredByYears = []
moviesByBudgetFilteredByYears = []
graphsFilteredByYears = []

for moviesData in moviesFilteredByYears:
    (adjacency, moviesByBudget) = ldata.make_budget_based_adjacency(moviesData);
    adjacenciesFilteredByYears.append(adjacency);
    moviesByBudgetFilteredByYears.append(moviesByBudget)
    graphsFilteredByYears.append(pg.graphs.Graph(adjacency))

In [ ]:
voteAvrgFtFilteredByYears = []
voteAvrgFilteredByYears = []

for i in range(len(graphsFilteredByYears)):
    voteAvrgFilteredByYears.append((moviesByBudgetFilteredByYears[i])['vote_average'].copy())
    (graphsFilteredByYears[i]).compute_fourier_basis()
    voteAvrgFtFilteredByYears.append(graphsFilteredByYears[i].gft((voteAvrgFilteredByYears[i]).values))

Now that we have filter our movies by decades, we can look at the correlation between the budget, the revenue, the vote average, the popularity and the return of investments. By plotting one correlation heatmap per decade, we can observe how the correlation evolved over the years.

In [ ]:
for i in range(len(graphsFilteredByYears)):
    decade = 1960 + i*10
    decadeEnd = decade + 10
    plt.figure(figsize=(5, 5))
    plt.title('Correlation heatmap, for the years {} to {}'.format(decade,decadeEnd))
    data= moviesByBudgetFilteredByYears[i].loc[:, ['budget', 'revenue','vote_average','popularity','ROI']]
    sns.heatmap(data=data.corr(),annot=True)

We also wanted to see if there was a more general correlation over the years between the revenue and the budget of movies and the global economy.
In the following section, we will try to visualize the **evolution in the average revenue** of the movies through years.

In [ ]:
nb_movies = movies['release_date'].values.shape[0]

#creating a matrix containing the movie's year and its revenue and budget.
rev_per_year = np.zeros((nb_movies,4))
for i in range(nb_movies):
    rev_per_year[i,0]=movies['release_date'].values[i].year
    rev_per_year[i,1]=movies['revenue'].values[i]
    rev_per_year[i,2]=movies['budget'].values[i]
    rev_per_year[i,3]=movies['ROI'].values[i]

years = np.unique(rev_per_year[:,0])
average_rev_per_year = np.zeros((years.shape[0],4))

#taking the average revenue of all the movies filmed in the same year
rev=0
bud=0
roi=0
for j in range(years.shape[0]):
    n = 0
    for i in range(nb_movies):
        if rev_per_year[i,0] == years[j] and ~np.isnan(rev_per_year[i,0]):
            rev = rev+rev_per_year[i,1]
            bud = bud+rev_per_year[i,2]
            roi = roi+rev_per_year[i,3]
            n = n+1
    average_rev_per_year[j,0]=years[j]
    average_rev_per_year[j,1]= rev/n
    average_rev_per_year[j,2]= bud/n
    average_rev_per_year[j,3]= roi/n

#removing the last nan row
average_rev_per_year=average_rev_per_year[:-2,:]

Now we can try to see if there is any correlation between the economy (here represented by the **S&P500** (index representing the 500 largest US companies) average yearly value) and the movies universe. The question we're trying to answer is if the economic crisis affect the movies industry or not.
The historical data of S&P 500 (index GSPC) will be recuperated via Yahoo finance.

In [ ]:
#The GSPC cvs file was recuperated via Yahoo finance
gspc=pd.read_csv('./data/GSPC.csv')
gspc_av_yearlyclose = np.zeros((2017-1950+1,2))
for i in range(gspc_av_yearlyclose.shape[0]):
    gspc_av_yearlyclose[i,0]=1950+i
    gspc_av_yearlyclose[i,1]=np.mean(gspc['Close'].values[12*(i+1)-12:12*(i+1)])

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Evolution of the revenue, the budget and the GSPC')
plt.xlabel('Year')
plt.ylabel('Dollars')
plt.plot(average_rev_per_year[:,0],average_rev_per_year[:,1], label='revenue')
plt.plot(average_rev_per_year[:,0],average_rev_per_year[:,2], label='budget')
plt.plot(gspc_av_yearlyclose[:,0],gspc_av_yearlyclose[:,1]*1000000, label='GSPC')
plt.legend(loc='upper left')
plt.show()

Now let's go back to the adjacency matrices of each decade
We now create the coordinates of the budget and revenue based graphs, and the coordinates of their giant components. Below are shown the 6 different adjacency matrices, the corresponding complete graphs and giant component graphs. We see that the number of movies increases with each deacde (with the exception of 2010-2020, since it is not over yet). As expected from such graphs based on distances, they have have long shapes.

In [ ]:
i = 0
GCgraphsFilteredByYears=[]
GgraphsFilteredByYears=[]
CoordsGcSpringFilteredByYears=[]
CoordsSpringFilteredByYears=[]

for G in graphsFilteredByYears:  
    #Creation of the coordinate of all the graphs and their giant components
    full_G = nx.from_numpy_matrix(adjacenciesFilteredByYears[i])
    GgraphsFilteredByYears.append(full_G)
    Gc = max(nx.connected_component_subgraphs(full_G), key=len)
    GCgraphsFilteredByYears.append(Gc)
    CoordsGcSpringFilteredByYears.append(nx.spring_layout(Gc,k=0.03))
    CoordsSpringFilteredByYears.append(nx.spring_layout(full_G,k=0.03))
    i = i+1

In [ ]:
for i in range(numberOfDecades):
    
    decade = 1960 + i*10
    decadeEnd = decade + 10
    
    adjacencyDecade=adjacenciesFilteredByYears[i]
    G_decade=GgraphsFilteredByYears[i]
    coords_decade=CoordsSpringFilteredByYears[i]
    
    Gc_decade=GCgraphsFilteredByYears[i]
    coords_decade_Gc=CoordsSpringFilteredByYears[i]
    
    plt.figure(figsize=(28, 7))
    plt.subplot(131)
    plt.spy(adjacencyDecade, markersize=1)
    plt.title('Adjacency for the years {} to {}'.format(decade, decadeEnd))   
    
    labels = moviesFilteredByYears[i]['genres'].iloc[np.sort(nx.nodes(G_decade))]
    plt.subplot(132)
    im=nx.draw_networkx_nodes(G_decade, coords_decade, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
    nx.draw_networkx_edges(G_decade, coords_decade, alpha=0.1, width=0.7)
    plt.title('Complete graph of common actors, with {} connected components'.format(nx.number_connected_components(G_decade)))
    plt.colorbar(im);
    
    nb_edges_gc=Gc_decade.number_of_edges()
    nb_nodes_gc=Gc_decade.number_of_nodes()
    labels = moviesFilteredByYears[i]['genres'].iloc[np.sort(nx.nodes(Gc_decade))]
    plt.subplot(133)
    im=nx.draw_networkx_nodes(Gc_decade, coords_decade_Gc, node_size=10,node_color=labels, cmap='tab20',vmin=0, vmax=20)
    nx.draw_networkx_edges(Gc_decade, coords_decade_Gc, alpha=0.1, width=0.7)
    plt.title('Giant component graph, with {} edges and {} nodes'.format(nb_edges_gc,nb_nodes_gc))
    plt.colorbar(im);

Then, by using these giant components graphs, we can plot different signals on them and compare them. Below are the budget in comparison with the movie genre. We can see that there are no communities formed by the movie genres.

In [ ]:
i = 0
for G in GCgraphsFilteredByYears:
    
    decade = 1960 + i*10
    decadeEnd = decade + 10
    
    plt.figure(figsize=(20, 8))
    
    plt.subplot(121)
    labels = moviesByBudgetFilteredByYears[i]['genres'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='tab20',vmin=0, vmax=20)
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Genre of movies for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
    
    plt.subplot(122)
    labels = moviesByBudgetFilteredByYears[i]['budget'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='PiYG')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Budget for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
        
    i=i+1
    

#### Here we can look at the budget and the revenue together.
This allows us to better observe the correlation that exist between them.

In [ ]:
i = 0
for G in GCgraphsFilteredByYears:
    
    plt.figure(figsize=(20, 8))
    plt.subplot(121)
    labels = moviesByBudgetFilteredByYears[i]['budget'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='PiYG')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Budget for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
    
    plt.subplot(122)
    labels = moviesByBudgetFilteredByYears[i]['revenue'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='PiYG')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Revenue for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
        
    i=i+1

#### Finaly we can look at the budget and the vote average.
We can see that the vote average is not dependent on the budget, as the ratings are similar in the entire graph.

In [ ]:
i = 0
for G in GCgraphsFilteredByYears:
    
    decade = 1960 + i*10
    decadeEnd = decade + 10
    
    plt.figure(figsize=(20, 8))
    plt.subplot(121)
    labels = moviesByBudgetFilteredByYears[i]['budget'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='PiYG')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Budget for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
    
    plt.subplot(122)
    labels = moviesByBudgetFilteredByYears[i]['vote_average'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='RdYlGn')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Vote average for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
        
    i=i+1

#### As we considered the success rate, we can also look at it next to the budget.
Here we can see that overall, movies with large budgets are all successfull, meaning that they are able to obtain a revenue of at least 150% of their budget. 

In [ ]:
i = 0
for G in GCgraphsFilteredByYears:
    
    decade = 1960 + i*10
    decadeEnd = decade + 10
    
    plt.figure(figsize=(20, 8))
    plt.subplot(121)
    labels = moviesByBudgetFilteredByYears[i]['budget'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='PiYG')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Budget for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
    
    plt.subplot(122)
    labels = moviesByBudgetFilteredByYears[i]['success'].iloc[np.sort(nx.nodes(G))]
    coords_spring=CoordsSpringFilteredByYears[i]
    im=nx.draw_networkx_nodes(G, coords_spring, node_size=10, node_color=labels, cmap='RdYlGn')
    nx.draw_networkx_edges(G, coords_spring, alpha=0.1, width=0.7)
    plt.title('Success rate for the years {} to {}'.format(decade,decadeEnd))
    plt.colorbar(im);
        
    i=i+1